# 📘 P1.3.4.3 – Database Queries from Web Endpoints
## Topic: Querying SQLite from Flask Web Endpoints

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Understand how to query a database from Flask endpoints
- Return query results as web responses
- Use different HTTP methods for database access

## What Does "Querying" Mean in SQL?

When you "fetch" data, you usually get all records from a table (like downloading the entire list).

**Querying** means asking the database for specific information using a question (query). You can:
- Select only certain columns
- Filter rows based on conditions (e.g., age > 18)
- Sort results
- Count, group, or join data

A query is not just about getting everything—it's about getting exactly what you need, using SQL statements like SELECT, WHERE, ORDER BY, GROUP BY, etc.

> In short: **Querying** is asking the database for specific, filtered, or processed data, not just fetching everything.

## 📝 Example
Flask makes it easy to query SQLite and send results to users.

In [ ]:
from flask import Flask, g, jsonify, request
import sqlite3
import os

app = Flask(__name__)
DB_FOLDER = os.path.join(os.path.dirname(__file__), 'db')
os.makedirs(DB_FOLDER, exist_ok=True)
DATABASE = os.path.join(DB_FOLDER, 'query.db')

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect(DATABASE)
    return g.db

# Create a table with multiple columns (run at startup)
def setup_db():
    with app.app_context():
        db = get_db()
        db.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                name TEXT,
                email TEXT,
                age INTEGER,
                city TEXT
            )
        ''')
        db.commit()
setup_db()

# Query users by different columns
@app.route('/users', methods=['GET'])
def get_users():
    db = get_db()
    query = 'SELECT * FROM users WHERE 1=1'
    params = []
    name = request.args.get('name')
    email = request.args.get('email')
    city = request.args.get('city')
    min_age = request.args.get('min_age')
    max_age = request.args.get('max_age')
    if name:
        query += ' AND name = ?'
        params.append(name)
    if email:
        query += ' AND email = ?'
        params.append(email)
    if city:
        query += ' AND city = ?'  #SELECT * FROM users WHERE 1=1 AND city = ?
        params.append(city)
    if min_age:
        query += ' AND age >= ?'
        params.append(int(min_age))
    if max_age:
        query += ' AND age <= ?'
        params.append(int(max_age))

    """query = "SELECT * FROM users WHERE 1=1 AND city = ?"
       params = ["London"]"""

    users = db.execute(query, params).fetchall() 
    return jsonify([
        {'id': row[0], 'name': row[1], 'email': row[2], 'age': row[3], 'city': row[4]} for row in users
    ])

@app.route('/add_user', methods=['POST'])
def add_user():
    db = get_db()
    data = request.json
    db.execute('''
        INSERT INTO users (name, email, age, city) VALUES (?, ?, ?, ?)
    ''', (
        data.get('name', 'Bob'),
        data.get('email', 'bob@example.com'),
        data.get('age', 25),
        data.get('city', 'Unknown')
    ))
    db.commit()
    return 'User added!'


**How to use these endpoints:**

- To add a user:
  - Route: `POST /add_user` (with JSON body)
  - Example curl:
 
```bash
curl -X POST http://localhost:5000/add_user -H "Content-Type: application/json" -d "{\"name\":\"Alice\",\"email\":\"alice@example.com\",\"age\":30,\"city\":\"London\"}"
```


- To get all users:
  - Route: `GET /users`
  - Example curl:
    ```bash
    curl http://localhost:5000/users

    name = None
    email = None
    city = None
    min_age = None
    max_age = None
    ```
```
Client (curl)
    │
    ▼
Flask Server (/users)
    │
    ▼
get_users()
    │
    ▼
request.args → None
    │
    ▼
get_db() → SQLite Connection
    │
    ▼
SQLite Query Execution (SELECT * FROM users WHERE 1=1)
    │
    ▼
Rows Fetched
    │
    ▼
jsonify()
    │
    ▼
HTTP JSON Response
```

- To query users by city:
  - Route: `GET /users?city=London`
  - Example curl:
    ```bash
    curl "http://localhost:5000/users?city=London"
    name = None
    email = None
    city = "London"
    min_age = None
    max_age = None
    ```
```
curl
  ↓
Flask Router
  ↓
get_users()
  ↓
request.args → city=London (SELECT * FROM users WHERE 1=1 AND city = ?)
  ↓
Build SQL with filter
  ↓
SQLite executes query
  ↓
Fetch filtered rows
  ↓
jsonify()
  ↓
HTTP Response
```

- To query users by age range:
  - Route: `GET /users?min_age=25&max_age=35`
  - Example curl:
    ```bash
    curl "http://localhost:5000/users?min_age=25&max_age=35"
    ```

- To query users by multiple fields:
  - Route: `GET /users?city=London&min_age=25`
  - Example curl:
    ```bash
    curl "http://localhost:5000/users?city=London&min_age=25"
    ```


## ✅ Key Takeaways
- Flask endpoints can query and return database data
- Results can be sent as JSON
- POST and GET methods are used for database access